# Transfer Learning

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [3]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [4]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([transforms.Grayscale(3),
                                 transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([transforms.Grayscale(3),
                               transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = "/content/drive/MyDrive/Stanford Files/Q15 Spring 2021/CS 229/Project/Data/processed_data"
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=2)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
dataiter = iter(dataloaders['train'])
images, labels = dataiter.next()

print(images.shape)
print(labels.shape)

torch.Size([4, 3, 32, 32])
torch.Size([4])


In [ ]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------
train Loss: 1.0971 Acc: 0.4947
val Loss: 0.9595 Acc: 0.5623

Epoch 1/24
----------
train Loss: 1.0125 Acc: 0.5138
val Loss: 0.6834 Acc: 0.6346

Epoch 2/24
----------
train Loss: 1.0452 Acc: 0.5276
val Loss: 1.2933 Acc: 0.5467

Epoch 3/24
----------
train Loss: 0.9076 Acc: 0.5188
val Loss: 0.9891 Acc: 0.5510

Epoch 4/24
----------
train Loss: 0.9399 Acc: 0.5390
val Loss: 0.7324 Acc: 0.6374

Epoch 5/24
----------
train Loss: 0.9468 Acc: 0.5418
val Loss: 0.9715 Acc: 0.4603

Epoch 6/24
----------
train Loss: 1.0258 Acc: 0.5255
val Loss: 1.8531 Acc: 0.5425

Epoch 7/24
----------
train Loss: 0.7023 Acc: 0.5748
val Loss: 1.2235 Acc: 0.6346

Epoch 8/24
----------
train Loss: 0.6894 Acc: 0.5769
val Loss: 0.7158 Acc: 0.6431

Epoch 9/24
----------
train Loss: 0.6839 Acc: 0.5914
val Loss: 1.0072 Acc: 0.5907

Epoch 10/24
----------
train Loss: 0.6796 Acc: 0.5982
val Loss: 1.0359 Acc: 0.6246

Epoch 11/24
----------
train Loss: 0.6801 Acc: 0.5950
val Loss: 0.6672 Acc: 0.6246

Ep

## 3 Images Stacked

In [9]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = "/content/drive/MyDrive/Stanford Files/Q15 Spring 2021/CS 229/Project/Data/processed_3months_data"
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=2)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [10]:
dataiter = iter(dataloaders['train'])
images, labels = dataiter.next()

print(images.shape)
print(labels.shape)

torch.Size([32, 3, 32, 32])
torch.Size([32])


In [11]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [12]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------
train Loss: 0.6991 Acc: 0.5268
val Loss: 102936.8970 Acc: 0.5498

Epoch 1/24
----------
train Loss: 0.6903 Acc: 0.5456
val Loss: 64084.6303 Acc: 0.4502

Epoch 2/24
----------
train Loss: 0.6892 Acc: 0.5470
val Loss: 170228.6061 Acc: 0.5498

Epoch 3/24
----------
train Loss: 0.6897 Acc: 0.5478
val Loss: 70667.8019 Acc: 0.4502

Epoch 4/24
----------
train Loss: 0.6895 Acc: 0.5470
val Loss: 5597.3155 Acc: 0.5498

Epoch 5/24
----------
train Loss: 0.6905 Acc: 0.5369
val Loss: 381027.7924 Acc: 0.5498

Epoch 6/24
----------
train Loss: 0.6908 Acc: 0.5456
val Loss: 8282.5237 Acc: 0.4502

Epoch 7/24
----------
train Loss: 0.6890 Acc: 0.5499
val Loss: 6621.2327 Acc: 0.4502

Epoch 8/24
----------
train Loss: 0.6888 Acc: 0.5499
val Loss: 31872.6909 Acc: 0.5498

Epoch 9/24
----------
train Loss: 0.6884 Acc: 0.5499
val Loss: 4599.1452 Acc: 0.5498

Epoch 10/24
----------
train Loss: 0.6888 Acc: 0.5499
val Loss: 4926.4610 Acc: 0.4502

Epoch 11/24
----------
train Loss: 0.6881 Ac